In [2]:
import numpy as np
import keras
import tensorflow as tf
from keras.layers import Input, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D, MaxPooling2D
from keras.models import Sequential
from keras.preprocessing import image
from keras.utils import layer_utils
from keras.utils.data_utils import get_file
from keras.applications.imagenet_utils import preprocess_input
from keras.initializers import glorot_uniform
import keras.backend as K
from matplotlib import pyplot
import scipy.misc
import cv2
from sklearn.model_selection import train_test_split

%matplotlib inline

K.set_image_data_format('channels_last')
K.set_learning_phase(1)

Using TensorFlow backend.


In [3]:
from os import listdir, walk
from os.path import isfile, join

mypath = "/Users/ayambabu/Documents/Machine Learning/lfw"
classes = [f for f in listdir(mypath) if not isfile(join(mypath, f))]
dictionary = {}
count = 1
num_triplets = 10000
samples = []

for i in classes:
    x = [f for f in listdir(mypath + "/" + i) if isfile(join(mypath + "/" + i, f))]
    dictionary[i] = x
    count = count + 1

for i in range(num_triplets):
    try:
        c_1, c_2 = np.random.choice(classes, size=2, replace=False)
        img_1, img_2 = np.random.choice(dictionary[c_1], size=2, replace=False)
        img_3 = np.random.choice(dictionary[c_2])
        triplet = (img_1, img_2, img_3)
        samples.append(triplet)
    except:
        i -= 1
    
#print(samples)

In [4]:
def jpgtofloat(img_path):
    im = cv2.imread(img_path)
    im2 = im
    
    #print(im.shape)
    
    faceCascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
    gray = cv2.cvtColor(im2, cv2.COLOR_BGR2GRAY)
    faces = faceCascade.detectMultiScale(gray, 1.3, 5) # (x, y, w, h)
    
    if len(faces) is not 0:
        #print(faces)
        face = faces[0]

        im[face[0] : (face[0] + face[2]), face[1] : (face[1] + face[3]), :]

        img = cv2.cvtColor(im, cv2.COLOR_BGR2RGB)

        return img
    else:
        return np.empty((0, 0, 0))

In [40]:
import pandas as pd

trainX = []
trainY = []

for c in classes:
    for item in dictionary[c]:
        arr = jpgtofloat(mypath + "/" + c + "/" + item)
        
        if(arr.shape is not (0,0,0)):
            trainX.append(arr)
            trainY.append(c)
        
dataset = pd.DataFrame({'X': trainX, 'y': trainY}, columns=['X', 'y'])
#dataset.to_excel(r"ImagesFR.xlsx", encoding="utf-8", index = None, header = True)

In [6]:
def getModel(f, s):
    model = Sequential()
    
    model.add(Conv2D(64, kernel_size = (f, f), strides = (s,s), padding = 'same', kernel_initializer = glorot_uniform(seed=0)))
    model.add(Conv2D(64, kernel_size = (f, f), strides = (s,s), padding = 'same', kernel_initializer = glorot_uniform(seed=0)))
    model.add(MaxPooling2D((f, f), strides=(s, s)))
    
    model.add(Conv2D(128, kernel_size = (f, f), strides = (s, s), padding = 'same', kernel_initializer = glorot_uniform(seed=0)))
    model.add(Conv2D(128, kernel_size = (f, f), strides = (s, s), padding = 'same', kernel_initializer = glorot_uniform(seed=0)))
    model.add(MaxPooling2D((f, f), strides=(s, s)))
    
    model.add(Conv2D(256, kernel_size = (f, f), strides = (s, s), padding = 'same', kernel_initializer = glorot_uniform(seed=0)))
    model.add(Conv2D(256, kernel_size = (f, f), strides = (s, s), padding = 'same', kernel_initializer = glorot_uniform(seed=0)))
    model.add(MaxPooling2D((f, f), strides=(s, s)))
    
    model.add(Conv2D(512, kernel_size = (f, f), strides = (s, s), padding = 'same', kernel_initializer = glorot_uniform(seed=0)))
    model.add(Conv2D(512, kernel_size = (f, f), strides = (s, s), padding = 'same', kernel_initializer = glorot_uniform(seed=0)))
    model.add(Conv2D(512, kernel_size = (f, f), strides = (s, s), padding = 'same', kernel_initializer = glorot_uniform(seed=0)))
    model.add(MaxPooling2D((f, f), strides=(s, s)))
    
    model.add(Conv2D(512, kernel_size = (f, f), strides = (s, s), padding = 'same', kernel_initializer = glorot_uniform(seed=0)))
    model.add(Conv2D(512, kernel_size = (f, f), strides = (s, s), padding = 'same', kernel_initializer = glorot_uniform(seed=0)))
    model.add(Conv2D(512, kernel_size = (f, f), strides = (s, s), padding = 'same', kernel_initializer = glorot_uniform(seed=0)))
    model.add(MaxPooling2D((f, f), strides=(s, s)))
    
    model.add(Flatten())
    model.add(Dense(4096, activation='softmax', kernel_initializer = glorot_uniform(seed=0)))
    model.add(Flatten())
    model.add(Dense(4096, activation='softmax', kernel_initializer = glorot_uniform(seed=0)))
    model.add(Flatten())
    model.add(Dense(2622, activation='softmax', kernel_initializer = glorot_uniform(seed=0)))
    
    return model

In [7]:
def triplet_loss(true, pred, alpha = 0.2):
    a, p, n = pred[0], pred[1], pred[2]
    
    pos_d = tf.reduce_sum((a - p)**2, axis=-1)
    neg_d = tf.reduce_sum((a - n)**2, axis=-1)
    b_loss = pos_d - neg_d + alpha
    loss = tf.reduce_sum(tf.maximum(b_loss, 0))
    
    return loss

In [28]:
df = pd.read_excel("/Users/ayambabu/Documents/Machine Learning/ImagesFR.xlsx", convert_float = True)
df

,X,y
0,[[[240 223 205]\n [240 223 205]\n [239 222 2...,German_Khan
1,[[[ 0 0 0]\n [ 0 0 0]\n [ 0 0 0]\n .....,Stefano_Gabbana
2,[[[117 112 108]\n [116 111 107]\n [115 110 1...,Dragan_Covic
3,[[[ 0 0 0]\n [ 0 0 0]\n [ 0 0 ...,Jeff_Hornacek
4,[[[ 3 0 0]\n [ 4 0 0]\n [ 4 0 ...,Sureyya_Ayhan
5,[[[ 47 28 30]\n [ 48 29 31]\n [ 49 30 ...,Deb_Santos
6,[[[ 0 1 4]\n [ 0 0 2]\n [ 1 2 ...,Bob_Newhart
7,[[[ 7 26 24]\n [ 7 26 24]\n [ 11 27 ...,Wang_Hailan
8,[[[ 10 21 25]\n [ 10 21 25]\n [ 10 19 ...,Paul_McNulty
9,[[[ 0 0 4]\n [ 1 1 3]\n [ 1 1 ...,Paul_McNulty


In [56]:
print(dataset["X"][0].shape)

(250, 250, 3)


In [68]:
from keras.preprocessing import image
from keras.applications.imagenet_utils import preprocess_input
from sklearn import preprocessing

def convert_to_one_hot(Y, C):
    Y = np.eye(C)[Y.reshape(-1)].T
    return Y

VGGFace = getModel(3, 1)
VGGFace.compile(optimizer = 'adam', loss = "triple_loss", metrics = ['accuracy'])

length = int(len(trainX)/2)

"""
trainXreal = np.array(trainX[0 : 9000])
testX = np.array(trainX[9000 : -1])
trainYreal = np.reshape(np.array(trainY[0 : 9000]), (9000, 1))
testY = np.reshape(np.array(trainY[9000 : -1]), (4232, 1))
"""

c = dataset["X"]
arr = c.as_matrix()

print(arr)

print(type(arr[0]))
print(arr.shape)

"""arr = np.empty((0, 0))

for i in range(len(dataset["X"])):
    arr = np.append(arr, dataset["X"][i])"""

np.reshape(arr, (13233, 250, 250, 3))

#print(np.fromstring(dataset["X"][0], int))

trainXreal, testX, trainYreal, testY = train_test_split(dataset["X"], dataset["y"], shuffle = True, test_size=0.2, random_state=42)

X_train = np.array(trainXreal) #/ 255.0
X_test = np.array(testX) #/ 255.0

print(X_train.shape)

print(type((trainYreal)))
print((np.array(trainYreal)))

le = preprocessing.LabelEncoder()
le.fit(np.array(classes))

Y_train = convert_to_one_hot(le.transform(np.array(trainYreal)), np.array(classes).shape[0]).T
Y_test = convert_to_one_hot(le.transform(np.array(testY)), np.array(classes).shape[0]).T

print(X_train.shape)
print(X_test.shape)
print(Y_train.shape)
print(Y_test.shape)

"""
try:
    VGGFace.fit(np.array(trainXreal), np.array(trainYreal), epochs = 10, batch_size = 100)
except:
    print(np.array(trainXreal).shape)
    print(np.array(testX).shape)
    print(np.array(trainYreal).shape)
    print(np.array(testY).shape)

preds = VGGFace.evaluate(np.array(testX), np.array(testY))
print ("Loss = " + str(preds[0]))
print ("Test Accuracy = " + str(preds[1]))

"""

[array([[[240, 223, 205],
        [240, 223, 205],
        [239, 222, 204],
        ...,
        [160,  81,  51],
        [159,  80,  50],
        [159,  80,  50]],

       [[237, 220, 202],
        [237, 220, 202],
        [237, 220, 202],
        ...,
        [160,  81,  51],
        [159,  80,  50],
        [159,  80,  50]],

       [[232, 215, 199],
        [233, 216, 200],
        [233, 216, 200],
        ...,
        [160,  81,  51],
        [159,  80,  50],
        [159,  80,  50]],

       ...,

       [[ 48,  36,  46],
        [ 48,  36,  46],
        [ 48,  36,  46],
        ...,
        [ 32,  23,  16],
        [ 35,  25,  23],
        [ 35,  25,  23]],

       [[ 51,  39,  51],
        [ 52,  40,  52],
        [ 51,  39,  51],
        ...,
        [ 30,  21,  14],
        [ 32,  22,  20],
        [ 32,  22,  20]],

       [[ 51,  39,  51],
        [ 52,  40,  52],
        [ 51,  39,  51],
        ...,
        [ 30,  21,  14],
        [ 31,  21,  19],
        [ 32,  22,  20]

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:22: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.


ValueError: cannot reshape array of size 13233 into shape (13233,250,250,3)

In [58]:
a = [1, 2, 3]
b = [4, 5, 6]

a = np.append(a, b)
print(a)

[1 2 3 4 5 6]


In [49]:
VGGFace.fit(np.array(trainXreal), np.reshape(np.array(trainYreal), np.array(trainYreal).shape[0]), epochs = 10, batch_size = 100)

ValueError: Input 0 is incompatible with layer conv2d_337: expected ndim=4, found ndim=1

In [77]:
from os import listdir, walk
from os.path import isfile, join

mypath = "/Users/ayambabu/Documents/Machine Learning/lfw"
classes = [f for f in listdir(mypath) if not isfile(join(mypath, f))]

dictionary = {}
num_triplets = 10000
samples = []

for i in classes:
    x = [f for f in listdir(mypath + "/" + i) if isfile(join(mypath + "/" + i, f))]
    dictionary[i] = x
    count = count + 1

for c in classes:
    for item in dictionary[c]:
        arr = jpgtofloat(mypath + "/" + c + "/" + item)
    
    image = cv2.imread(imagePath)
    rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)boxes = face_recognition.face_locations(rgb, model=args["detection_method"])
    
    boxes = face_recognition.face_locations(rgb, model=args["detection_method"])
    encodings = face_recognition.face_encodings(rgb, boxes)

    for encoding in encodings:
        knownEncodings.append(encoding)
        knownNames.append(c)

        df = {"encodings":knownEncodings, "names":knownNames}

usage: ipykernel_launcher.py [-h] -e ENCODINGS [-o OUTPUT] [-y DISPLAY]
                             [-d DETECTION_METHOD]
ipykernel_launcher.py: error: the following arguments are required: -e/--encodings


SystemExit: 2

/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3304: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [73]:
def jpgtofloat(img_path):
    im = cv2.imread(img_path)
    #im2 = im
    
    #print(im.shape)
    
    faces = face_recognition.face_locations(rgb, model=args["detection_method"])
    
    if len(faces) is not 0:
        #print(faces)
        face = faces[0]

        #im[face[0] : (face[0] + face[2]), face[1] : (face[1] + face[3]), :]

        img = cv2.cvtColor(im, cv2.COLOR_BGR2RGB)

        return rgb, faces, img
    else:
        return 0, 0, np.empty((0, 0, 0))